In [ ]:
# 필요한 라이브러리 설치 및 임포트
import subprocess
import sys

def install_package(package):
    """필요한 패키지를 자동으로 설치하는 함수"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} 설치 완료")
    except subprocess.CalledProcessError:
        print(f"❌ {package} 설치 실패")

# 필요한 패키지들 설치
required_packages = ['pandas', 'requests', 'beautifulsoup4', 'matplotlib', 'seaborn', 'wordcloud', 'numpy']

for package in required_packages:
    try:
        __import__(package.replace('-', '_'))
        print(f"✅ {package} 이미 설치됨")
    except ImportError:
        print(f"📦 {package} 설치 중...")
        install_package(package)


In [ ]:
# 라이브러리 임포트
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
plt.rcParams['axes.unicode_minus'] = False

# seaborn과 wordcloud는 선택적 임포트 (설치되지 않은 경우 대체 방법 사용)
try:
    import seaborn as sns
    sns.set_style("whitegrid")
    print("✅ seaborn 임포트 성공")
except ImportError:
    print("⚠️ seaborn을 사용할 수 없습니다. matplotlib만 사용합니다.")
    sns = None

try:
    from wordcloud import WordCloud
    print("✅ wordcloud 임포트 성공")
except ImportError:
    print("⚠️ wordcloud를 사용할 수 없습니다. 대체 시각화를 사용합니다.")
    WordCloud = None

print("📚 모든 라이브러리 임포트 완료!")


In [ ]:
# 2.3 실제 전통시장 데이터 로드
# 공공데이터포털에서 다운로드한 실제 데이터를 사용합니다.

# 실제 전통시장 매칭 데이터 로드
try:
    market_df = pd.read_csv('전통시장_매칭데이터.csv', encoding='utf-8')
    print("✅ 실제 전통시장 데이터 로드 성공!")
except:
    try:
        market_df = pd.read_csv('전통시장_매칭데이터.csv', encoding='cp949')
        print("✅ 실제 전통시장 데이터 로드 성공! (cp949 인코딩)")
    except Exception as e:
        print(f"❌ 데이터 로드 실패: {e}")
        # 백업 데이터 생성
        market_df = pd.DataFrame()

print(f"📊 총 {len(market_df)}개 전통시장 데이터")
print(f"📋 컬럼 수: {len(market_df.columns)}개")
print("\n🔍 데이터 기본 정보:")
print(market_df.info())

print("\n📋 주요 컬럼들:")
print(market_df.columns.tolist()[:10])  # 처음 10개 컬럼만 출력


In [ ]:
# 2.4 데이터 전처리 및 정제
# 실제 데이터의 컬럼명을 정리하고 분석에 필요한 형태로 변환

def preprocess_market_data(df):
    """전통시장 데이터 전처리 함수"""
    
    if df.empty:
        print("❌ 데이터가 비어있습니다.")
        return df
    
    # 주요 컬럼 추출 및 이름 변경
    processed_df = df.copy()
    
    # 컬럼명 매핑 (실제 데이터 구조에 맞게)
    column_mapping = {
        '시장명_정제': '시장명',
        '시도_파일1': '시도',
        '시군구_파일1': '시군구', 
        '점포수_파일2': '점포수',
        '아케이드 보유 여부_파일1': '아케이드',
        '시장전용 고객주차장_보유여부_파일1': '주차장',
        '공중화장실보유여부_파일2': '화장실',
        '개설연도_파일2': '개설연도',
        '취급품목_파일2': '취급품목',
        '시장유형_파일2': '시장유형'
    }
    
    # 존재하는 컬럼만 매핑
    existing_columns = {}
    for old_col, new_col in column_mapping.items():
        if old_col in processed_df.columns:
            existing_columns[old_col] = new_col
    
    processed_df = processed_df.rename(columns=existing_columns)
    
    # 데이터 타입 변환 및 정제
    if '점포수' in processed_df.columns:
        processed_df['점포수'] = pd.to_numeric(processed_df['점포수'], errors='coerce')
    
    if '개설연도' in processed_df.columns:
        processed_df['개설연도'] = pd.to_numeric(processed_df['개설연도'], errors='coerce')
        processed_df['운영년수'] = 2024 - processed_df['개설연도']
    
    # Y/N 값을 한글로 변환
    yn_columns = ['아케이드', '주차장', '화장실']
    for col in yn_columns:
        if col in processed_df.columns:
            processed_df[col] = processed_df[col].map({'Y': '있음', 'N': '없음'}).fillna('정보없음')
    
    print("✅ 데이터 전처리 완료!")
    return processed_df

# 데이터 전처리 실행
market_df_processed = preprocess_market_data(market_df)

print("\n📊 전처리된 데이터 미리보기:")
if not market_df_processed.empty:
    print(market_df_processed[['시장명', '시도', '점포수', '아케이드', '주차장']].head())
else:
    print("데이터가 없습니다.")


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
# numpy는 시뮬레이션 데이터 생성에만 필요하므로 제거합니다.

# 2.5 리뷰 데이터 크롤링 함수 정의
# (네이버 지도 크롤링은 동적 콘텐츠로 인해 requests+BeautifulSoup만으로는 한계가 있습니다.
#  실제 동작을 위해서는 Selenium과 같은 headless browser 라이브러리가 필요합니다.
#  여기서는 requests+BeautifulSoup를 사용한 기본적인 시도를 보여주며, 데이터 추출이 어려울 수 있습니다.)

def get_review_data(market_name):
    """
    네이버 지도에서 시장 리뷰 데이터를 크롤링 시도.
    동적 콘텐츠로 인해 requests + BeautifulSoup만으로는 정확한 평점/리뷰수 추출이 어려울 수 있습니다.
    실제 크롤링 시에는 Selenium과 같은 도구를 사용하여 JavaScript가 렌더링된 후의 HTML을 분석해야 합니다.
    """
    base_url = "https://map.naver.com/v5/search/"
    # 검색 정확도를 높이기 위해 시장명 뒤에 '전통시장'을 추가합니다.
    search_query = f"{market_name} 전통시장" 
    url = f"{base_url}{search_query}"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    rating = 0.0 # 크롤링 실패 시 기본값
    review_count = 0 # 크롤링 실패 시 기본값
    
    try:
        # HTTP 요청을 보내고 응답을 받습니다. 타임아웃 설정으로 무한 대기 방지.
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status() # 4xx, 5xx 에러 발생 시 예외 처리
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # --- 중요: 네이버 지도와 같은 동적 웹페이지 크롤링의 한계 ---
        # requests+BeautifulSoup는 웹페이지의 정적 HTML만을 가져옵니다.
        # 네이버 지도의 평점, 리뷰 수 등은 JavaScript에 의해 동적으로 로드되므로,
        # 이 방법으로는 해당 데이터를 직접 추출하기 매우 어렵습니다.
        # 아래의 CSS 선택자는 예시이며, 실제로는 작동하지 않을 가능성이 높습니다.
        # 실제 데이터를 얻으려면 Selenium과 같은 headless browser 라이브러리가 필수적입니다.
        # ----------------------------------------------------------

        # 예시: 평점과 리뷰수를 포함할 수 있는 요소 탐색 (실제로는 찾기 어려움)
        # 이 부분은 실제 네이버 지도 페이지의 HTML 소스를 보고 작성해야 합니다.
        # 현재 requests+BeautifulSoup로는 동적 로딩되는 데이터를 가져오기 어렵기 때문에,
        # 대부분의 경우 기본값(0.0, 0)이 반환될 것입니다.
        
        # 평점 추출 시도 (예시 선택자, 실제와 다를 수 있음)
        rating_element = soup.find('span', class_='_score') 
        if rating_element:
            try:
                rating = float(rating_element.text.strip())
            except ValueError:
                pass
        
        # 리뷰수 추출 시도 (예시 선택자, 실제와 다를 수 있음)
        review_count_element = soup.find('span', class_='_reviewCount') 
        if review_count_element:
            try:
                review_count_text = review_count_element.text.strip().replace(',', '')
                review_count = int(review_count_text.strip('()'))
            except ValueError:
                pass

        if rating == 0.0 and review_count == 0:
            print(f"⚠️ '{market_name}'에 대한 리뷰 데이터를 찾기 어렵습니다. (동적 콘텐츠 문제 또는 데이터 없음)")
            
    except requests.exceptions.RequestException as e:
        print(f"❌ 크롤링 오류 발생 ({market_name}): {e}")
    except Exception as e:
        print(f"❌ 데이터 파싱 오류 ({market_name}): {e}")
        
    return rating, review_count

def get_market_reviews(market_df):
    """전통시장별 리뷰 데이터를 크롤링하여 데이터프레임에 추가"""
    
    if market_df.empty:
        print("❌ 데이터프레임이 비어있어 리뷰 데이터를 크롤링할 수 없습니다.")
        return market_df
    
    print("🕷️ 리뷰 데이터 크롤링 시작...")
    
    ratings = []
    review_counts = []
    
    # 실제 크롤링 시에는 서버에 과도한 부하를 주지 않기 위해 충분한 지연 시간을 두어야 합니다.
    CRAWL_DELAY = 1.0 # 실제 크롤링이므로 지연 시간을 늘림 (초 단위)
    
    for idx, row in market_df.iterrows():
        market_name = row.get('시장명', f'시장_{idx}')
        
        # 리뷰 데이터 크롤링 시도
        rating, review_count = get_review_data(market_name)
        
        ratings.append(rating)
        review_counts.append(review_count)
        
        # 진행 상황 표시 및 지연 시간
        time.sleep(CRAWL_DELAY)
        
        # 10개 시장마다 또는 마지막 시장일 때 진행률 출력
        if (idx + 1) % 10 == 0 or (idx + 1) == len(market_df):
            print(f"진행률: {idx+1}/{len(market_df)} ({(idx+1)/len(market_df)*100:.1f}%) - 현재 시장: {market_name}")
    
    # 크롤링된 리뷰 데이터를 원본 데이터프레임에 추가
    market_df_with_reviews = market_df.copy()
    market_df_with_reviews['평점'] = ratings
    market_df_with_reviews['리뷰수'] = review_counts
    
    print("✅ 리뷰 데이터 크롤링 완료!")
    return market_df_with_reviews

# 리뷰 데이터 생성 (크롤링 함수 호출)
market_df_final = get_market_reviews(market_df_processed)


In [ ]:
# 3.4 기본 통계 분석
# 전통시장 현황에 대한 기술통계 분석

def analyze_basic_statistics(df):
    """기본 통계 분석 함수"""
    
    if df.empty:
        print("❌ 분석할 데이터가 없습니다.")
        return
    
    print("📊 전국 전통시장 현황 기본 통계")
    print("=" * 50)
    
    # 전체 시장 수
    total_markets = len(df)
    print(f"🏪 총 전통시장 수: {total_markets:,}개")
    
    # 지역별 분포
    if '시도' in df.columns:
        region_dist = df['시도'].value_counts()
        print(f"\n📍 지역별 시장 분포 (상위 5개 지역):")
        for region, count in region_dist.head().items():
            print(f"   {region}: {count}개 ({count/total_markets*100:.1f}%)")
    
    # 점포수 통계
    if '점포수' in df.columns:
        store_stats = df['점포수'].describe()
        print(f"\n🏬 점포수 통계:")
        print(f"   평균: {store_stats['mean']:.1f}개")
        print(f"   중앙값: {store_stats['50%']:.1f}개")
        print(f"   최대: {store_stats['max']:.0f}개")
        print(f"   최소: {store_stats['min']:.0f}개")
    
    # 시설 보유율
    facility_columns = ['아케이드', '주차장', '화장실']
    print(f"\n🏗️ 시설 보유율:")
    for col in facility_columns:
        if col in df.columns:
            has_facility = (df[col] == '있음').sum()
            rate = has_facility / total_markets * 100
            print(f"   {col}: {has_facility}개 ({rate:.1f}%)")
    
    # 평점 및 리뷰 통계
    if '평점' in df.columns:
        rating_stats = df['평점'].describe()
        print(f"\n⭐ 평점 통계:")
        print(f"   평균 평점: {rating_stats['mean']:.2f}점")
        print(f"   최고 평점: {rating_stats['max']:.1f}점")
        print(f"   최저 평점: {rating_stats['min']:.1f}점")
    
    if '리뷰수' in df.columns:
        review_stats = df['리뷰수'].describe()
        print(f"\n💬 리뷰수 통계:")
        print(f"   평균 리뷰수: {review_stats['mean']:.1f}개")
        print(f"   최대 리뷰수: {review_stats['max']:.0f}개")
    
    # 운영년수 통계
    if '운영년수' in df.columns:
        age_stats = df['운영년수'].describe()
        print(f"\n📅 운영년수 통계:")
        print(f"   평균 운영년수: {age_stats['mean']:.1f}년")
        print(f"   최장 운영: {age_stats['max']:.0f}년")
        print(f"   최단 운영: {age_stats['min']:.0f}년")

# 기본 통계 분석 실행
analyze_basic_statistics(market_df_final)


In [ ]:
# 3.5 시각화 분석
# 다양한 차트를 통한 전통시장 현황 시각화

def create_visualizations(df):
    """전통시장 데이터 시각화 함수"""
    
    if df.empty:
        print("❌ 시각화할 데이터가 없습니다.")
        return
    
    # 그래프 스타일 설정
    plt.style.use('default')
    fig = plt.figure(figsize=(20, 15))
    
    # 1. 지역별 시장 분포
    if '시도' in df.columns:
        plt.subplot(3, 3, 1)
        region_counts = df['시도'].value_counts().head(10)
        plt.bar(range(len(region_counts)), region_counts.values, color='skyblue')
        plt.title('지역별 전통시장 분포 (상위 10개)', fontsize=12, fontweight='bold')
        plt.xlabel('지역')
        plt.ylabel('시장 수')
        plt.xticks(range(len(region_counts)), region_counts.index, rotation=45)
        
        # 값 표시
        for i, v in enumerate(region_counts.values):
            plt.text(i, v + 5, str(v), ha='center', va='bottom')
    
    # 2. 점포수 분포
    if '점포수' in df.columns:
        plt.subplot(3, 3, 2)
        valid_stores = df['점포수'].dropna()
        plt.hist(valid_stores, bins=30, color='lightgreen', alpha=0.7, edgecolor='black')
        plt.title('점포수 분포', fontsize=12, fontweight='bold')
        plt.xlabel('점포수')
        plt.ylabel('시장 수')
        plt.axvline(valid_stores.mean(), color='red', linestyle='--', 
                   label=f'평균: {valid_stores.mean():.1f}개')
        plt.legend()
    
    # 3. 시설 보유율
    plt.subplot(3, 3, 3)
    facility_data = []
    facility_labels = []
    
    for facility in ['아케이드', '주차장', '화장실']:
        if facility in df.columns:
            has_facility = (df[facility] == '있음').sum()
            rate = has_facility / len(df) * 100
            facility_data.append(rate)
            facility_labels.append(facility)
    
    if facility_data:
        colors = ['#ff9999', '#66b3ff', '#99ff99']
        plt.bar(facility_labels, facility_data, color=colors[:len(facility_data)])
        plt.title('시설 보유율', fontsize=12, fontweight='bold')
        plt.ylabel('보유율 (%)')
        
        # 값 표시
        for i, v in enumerate(facility_data):
            plt.text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom')
    
    # 4. 평점 분포
    if '평점' in df.columns:
        plt.subplot(3, 3, 4)
        valid_ratings = df['평점'].dropna()
        plt.hist(valid_ratings, bins=20, color='gold', alpha=0.7, edgecolor='black')
        plt.title('평점 분포', fontsize=12, fontweight='bold')
        plt.xlabel('평점')
        plt.ylabel('시장 수')
        plt.axvline(valid_ratings.mean(), color='red', linestyle='--',
                   label=f'평균: {valid_ratings.mean():.2f}점')
        plt.legend()
    
    # 5. 리뷰수 분포
    if '리뷰수' in df.columns:
        plt.subplot(3, 3, 5)
        valid_reviews = df['리뷰수'].dropna()
        plt.hist(valid_reviews, bins=30, color='orange', alpha=0.7, edgecolor='black')
        plt.title('리뷰수 분포', fontsize=12, fontweight='bold')
        plt.xlabel('리뷰수')
        plt.ylabel('시장 수')
        plt.axvline(valid_reviews.mean(), color='red', linestyle='--',
                   label=f'평균: {valid_reviews.mean():.1f}개')
        plt.legend()
    
    # 6. 점포수 vs 평점 산점도
    if '점포수' in df.columns and '평점' in df.columns:
        plt.subplot(3, 3, 6)
        valid_data = df[['점포수', '평점']].dropna()
        plt.scatter(valid_data['점포수'], valid_data['평점'], alpha=0.6, color='purple')
        plt.title('점포수 vs 평점', fontsize=12, fontweight='bold')
        plt.xlabel('점포수')
        plt.ylabel('평점')
        
        # 추세선 추가
        if len(valid_data) > 1:
            z = np.polyfit(valid_data['점포수'], valid_data['평점'], 1)
            p = np.poly1d(z)
            plt.plot(valid_data['점포수'], p(valid_data['점포수']), "r--", alpha=0.8)
    
    # 7. 운영년수 분포
    if '운영년수' in df.columns:
        plt.subplot(3, 3, 7)
        valid_age = df['운영년수'].dropna()
        valid_age = valid_age[valid_age >= 0]  # 음수 제거
        plt.hist(valid_age, bins=25, color='lightcoral', alpha=0.7, edgecolor='black')
        plt.title('운영년수 분포', fontsize=12, fontweight='bold')
        plt.xlabel('운영년수')
        plt.ylabel('시장 수')
        plt.axvline(valid_age.mean(), color='red', linestyle='--',
                   label=f'평균: {valid_age.mean():.1f}년')
        plt.legend()
    
    # 8. 시장유형별 분포
    if '시장유형' in df.columns:
        plt.subplot(3, 3, 8)
        market_types = df['시장유형'].value_counts()
        plt.pie(market_types.values, labels=market_types.index, autopct='%1.1f%%',
                startangle=90, colors=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99'])
        plt.title('시장유형별 분포', fontsize=12, fontweight='bold')
    
    # 9. 평점 vs 리뷰수 산점도
    if '평점' in df.columns and '리뷰수' in df.columns:
        plt.subplot(3, 3, 9)
        valid_data = df[['평점', '리뷰수']].dropna()
        plt.scatter(valid_data['평점'], valid_data['리뷰수'], alpha=0.6, color='green')
        plt.title('평점 vs 리뷰수', fontsize=12, fontweight='bold')
        plt.xlabel('평점')
        plt.ylabel('리뷰수')
        
        # 추세선 추가
        if len(valid_data) > 1:
            z = np.polyfit(valid_data['평점'], valid_data['리뷰수'], 1)
            p = np.poly1d(z)
            plt.plot(valid_data['평점'], p(valid_data['평점']), "r--", alpha=0.8)
    
    plt.tight_layout()
    plt.show()
    
    print("📈 시각화 분석 완료!")

# 시각화 실행
create_visualizations(market_df_final)


In [ ]:
# 3.6 상관관계 분석
# 시설현황과 고객 만족도 간의 관계 분석

def correlation_analysis(df):
    """상관관계 분석 함수"""
    
    if df.empty:
        print("❌ 분석할 데이터가 없습니다.")
        return
    
    print("🔍 시설현황과 고객 만족도 상관관계 분석")
    print("=" * 50)
    
    # 수치형 변수들 선택
    numeric_columns = []
    
    # 시설 보유 여부를 수치형으로 변환
    facility_mapping = {'있음': 1, '없음': 0, '정보없음': 0}
    
    for col in ['아케이드', '주차장', '화장실']:
        if col in df.columns:
            df[f'{col}_수치'] = df[col].map(facility_mapping)
            numeric_columns.append(f'{col}_수치')
    
    # 기존 수치형 컬럼들 추가
    for col in ['점포수', '평점', '리뷰수', '운영년수']:
        if col in df.columns:
            numeric_columns.append(col)
    
    if len(numeric_columns) < 2:
        print("❌ 상관관계 분석을 위한 충분한 수치형 데이터가 없습니다.")
        return
    
    # 상관관계 계산
    correlation_data = df[numeric_columns].corr()
    
    print("📊 주요 상관관계:")
    
    # 평점과 다른 변수들의 상관관계
    if '평점' in correlation_data.columns:
        rating_corr = correlation_data['평점'].sort_values(ascending=False)
        print("\\n⭐ 평점과의 상관관계:")
        for var, corr in rating_corr.items():
            if var != '평점':
                print(f"   {var}: {corr:.3f}")
    
    # 리뷰수와 다른 변수들의 상관관계
    if '리뷰수' in correlation_data.columns:
        review_corr = correlation_data['리뷰수'].sort_values(ascending=False)
        print("\\n💬 리뷰수와의 상관관계:")
        for var, corr in review_corr.items():
            if var != '리뷰수':
                print(f"   {var}: {corr:.3f}")
    
    # 상관관계 히트맵 시각화
    plt.figure(figsize=(12, 10))
    
    # seaborn이 있으면 사용, 없으면 matplotlib로 대체
    if 'sns' in globals() and sns is not None:
        sns.heatmap(correlation_data, annot=True, cmap='coolwarm', center=0,
                   square=True, fmt='.3f', cbar_kws={'shrink': 0.8})
    else:
        # matplotlib로 히트맵 구현
        im = plt.imshow(correlation_data.values, cmap='coolwarm', aspect='auto')
        plt.colorbar(im, shrink=0.8)
        
        # 축 레이블 설정
        plt.xticks(range(len(correlation_data.columns)), correlation_data.columns, rotation=45)
        plt.yticks(range(len(correlation_data.columns)), correlation_data.columns)
        
        # 상관계수 값 표시
        for i in range(len(correlation_data.columns)):
            for j in range(len(correlation_data.columns)):
                plt.text(j, i, f'{correlation_data.iloc[i, j]:.3f}',
                        ha='center', va='center', color='black')
    
    plt.title('시설현황과 고객 만족도 상관관계 히트맵', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # 강한 상관관계 찾기
    print("\\n🔥 강한 상관관계 (|r| > 0.3):")
    strong_correlations = []
    
    for i in range(len(correlation_data.columns)):
        for j in range(i+1, len(correlation_data.columns)):
            corr_value = correlation_data.iloc[i, j]
            if abs(corr_value) > 0.3:
                var1 = correlation_data.columns[i]
                var2 = correlation_data.columns[j]
                strong_correlations.append((var1, var2, corr_value))
    
    if strong_correlations:
        for var1, var2, corr in sorted(strong_correlations, key=lambda x: abs(x[2]), reverse=True):
            direction = "양의" if corr > 0 else "음의"
            print(f"   {var1} ↔ {var2}: {corr:.3f} ({direction} 상관관계)")
    else:
        print("   강한 상관관계를 보이는 변수 쌍이 없습니다.")

# 상관관계 분석 실행
correlation_analysis(market_df_final)


In [ ]:
# 4.2 심화 분석 - 고성과 시장 vs 저성과 시장 비교
# 평점을 기준으로 상위/하위 시장들의 특성 비교

def analyze_high_low_performance(df):
    """고성과 vs 저성과 시장 비교 분석"""
    
    if df.empty or '평점' not in df.columns:
        print("❌ 성과 분석을 위한 데이터가 없습니다.")
        return
    
    print("🏆 고성과 vs 저성과 시장 비교 분석")
    print("=" * 50)
    
    # 평점 기준으로 상위 20%, 하위 20% 시장 선별
    valid_ratings = df['평점'].dropna()
    top_20_threshold = valid_ratings.quantile(0.8)
    bottom_20_threshold = valid_ratings.quantile(0.2)
    
    high_performance = df[df['평점'] >= top_20_threshold]
    low_performance = df[df['평점'] <= bottom_20_threshold]
    
    print(f"📊 분석 대상:")
    print(f"   고성과 시장 (상위 20%): {len(high_performance)}개 (평점 {top_20_threshold:.1f}점 이상)")
    print(f"   저성과 시장 (하위 20%): {len(low_performance)}개 (평점 {bottom_20_threshold:.1f}점 이하)")
    
    # 시설 보유율 비교
    print(f"\\n🏗️ 시설 보유율 비교:")
    facilities = ['아케이드', '주차장', '화장실']
    
    for facility in facilities:
        if facility in df.columns:
            high_rate = (high_performance[facility] == '있음').mean() * 100
            low_rate = (low_performance[facility] == '있음').mean() * 100
            diff = high_rate - low_rate
            
            print(f"   {facility}:")
            print(f"     고성과: {high_rate:.1f}% | 저성과: {low_rate:.1f}% | 차이: {diff:+.1f}%p")
    
    # 점포수 비교
    if '점포수' in df.columns:
        high_stores = high_performance['점포수'].mean()
        low_stores = low_performance['점포수'].mean()
        print(f"\\n🏬 평균 점포수:")
        print(f"   고성과: {high_stores:.1f}개 | 저성과: {low_stores:.1f}개")
    
    # 운영년수 비교
    if '운영년수' in df.columns:
        high_age = high_performance['운영년수'].mean()
        low_age = low_performance['운영년수'].mean()
        print(f"\\n📅 평균 운영년수:")
        print(f"   고성과: {high_age:.1f}년 | 저성과: {low_age:.1f}년")
    
    # 지역별 분포 비교
    if '시도' in df.columns:
        print(f"\\n📍 지역별 분포 (상위 5개 지역):")
        print("   고성과 시장:")
        high_regions = high_performance['시도'].value_counts().head()
        for region, count in high_regions.items():
            rate = count / len(high_performance) * 100
            print(f"     {region}: {count}개 ({rate:.1f}%)")
        
        print("   저성과 시장:")
        low_regions = low_performance['시도'].value_counts().head()
        for region, count in low_regions.items():
            rate = count / len(low_performance) * 100
            print(f"     {region}: {count}개 ({rate:.1f}%)")
    
    # 시각화
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. 시설 보유율 비교
    ax1 = axes[0, 0]
    facility_comparison = []
    labels = []
    
    for facility in facilities:
        if facility in df.columns:
            high_rate = (high_performance[facility] == '있음').mean() * 100
            low_rate = (low_performance[facility] == '있음').mean() * 100
            facility_comparison.append([high_rate, low_rate])
            labels.append(facility)
    
    if facility_comparison:
        x = np.arange(len(labels))
        width = 0.35
        
        high_rates = [item[0] for item in facility_comparison]
        low_rates = [item[1] for item in facility_comparison]
        
        ax1.bar(x - width/2, high_rates, width, label='고성과', color='lightgreen')
        ax1.bar(x + width/2, low_rates, width, label='저성과', color='lightcoral')
        
        ax1.set_xlabel('시설 유형')
        ax1.set_ylabel('보유율 (%)')
        ax1.set_title('시설 보유율 비교')
        ax1.set_xticks(x)
        ax1.set_xticklabels(labels)
        ax1.legend()
        ax1.grid(True, alpha=0.3)
    
    # 2. 점포수 분포 비교
    ax2 = axes[0, 1]
    if '점포수' in df.columns:
        high_stores = high_performance['점포수'].dropna()
        low_stores = low_performance['점포수'].dropna()
        
        ax2.hist(high_stores, bins=20, alpha=0.7, label='고성과', color='lightgreen')
        ax2.hist(low_stores, bins=20, alpha=0.7, label='저성과', color='lightcoral')
        ax2.set_xlabel('점포수')
        ax2.set_ylabel('시장 수')
        ax2.set_title('점포수 분포 비교')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    # 3. 평점 분포
    ax3 = axes[1, 0]
    ax3.hist(high_performance['평점'], bins=15, alpha=0.7, label='고성과', color='lightgreen')
    ax3.hist(low_performance['평점'], bins=15, alpha=0.7, label='저성과', color='lightcoral')
    ax3.set_xlabel('평점')
    ax3.set_ylabel('시장 수')
    ax3.set_title('평점 분포')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. 리뷰수 분포
    ax4 = axes[1, 1]
    if '리뷰수' in df.columns:
        high_reviews = high_performance['리뷰수'].dropna()
        low_reviews = low_performance['리뷰수'].dropna()
        
        ax4.hist(high_reviews, bins=20, alpha=0.7, label='고성과', color='lightgreen')
        ax4.hist(low_reviews, bins=20, alpha=0.7, label='저성과', color='lightcoral')
        ax4.set_xlabel('리뷰수')
        ax4.set_ylabel('시장 수')
        ax4.set_title('리뷰수 분포 비교')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return high_performance, low_performance

# 고성과 vs 저성과 분석 실행
high_perf_markets, low_perf_markets = analyze_high_low_performance(market_df_final)


In [ ]:
# 4.3 키워드 분석 및 워드클라우드
# 리뷰 키워드 분석 시뮬레이션

def keyword_analysis_simulation(df):
    """키워드 분석 시뮬레이션 함수"""
    
    print("🔤 전통시장 리뷰 키워드 분석 (시뮬레이션)")
    print("=" * 50)
    
    # 전통시장 관련 키워드 시뮬레이션 데이터
    positive_keywords = [
        '맛있다', '신선하다', '저렴하다', '친절하다', '깨끗하다', '다양하다',
        '전통적이다', '정겨웁다', '활기차다', '편리하다', '좋다', '만족하다',
        '추천한다', '재방문', '품질좋다', '가성비', '분위기좋다', '접근성좋다'
    ]
    
    negative_keywords = [
        '비싸다', '불친절하다', '더럽다', '오래되다', '불편하다', '시끄럽다',
        '주차어렵다', '냄새난다', '좁다', '복잡하다', '아쉽다', '개선필요',
        '화장실더럽다', '접근어렵다', '시설낡다', '관리부족'
    ]
    
    facility_keywords = [
        '주차장', '화장실', '아케이드', '에어컨', '난방', '조명', '안전',
        '접근성', '교통', '지하철', '버스', '택시', '엘리베이터', '계단'
    ]
    
    # 키워드 빈도 시뮬레이션 (평점에 따라 가중치 적용)
    np.random.seed(42)
    
    keyword_freq = {}
    
    # 긍정 키워드 (평점이 높을수록 빈도 증가)
    for keyword in positive_keywords:
        base_freq = np.random.randint(50, 200)
        # 평점 평균에 따라 가중치 적용
        if '평점' in df.columns:
            avg_rating = df['평점'].mean()
            weight = avg_rating / 5.0
            keyword_freq[keyword] = int(base_freq * weight)
        else:
            keyword_freq[keyword] = base_freq
    
    # 부정 키워드 (평점이 낮을수록 빈도 증가)
    for keyword in negative_keywords:
        base_freq = np.random.randint(20, 80)
        if '평점' in df.columns:
            avg_rating = df['평점'].mean()
            weight = (5.0 - avg_rating) / 5.0
            keyword_freq[keyword] = int(base_freq * weight)
        else:
            keyword_freq[keyword] = base_freq
    
    # 시설 관련 키워드
    for keyword in facility_keywords:
        keyword_freq[keyword] = np.random.randint(30, 120)
    
    # 상위 키워드 출력
    sorted_keywords = sorted(keyword_freq.items(), key=lambda x: x[1], reverse=True)
    
    print("📊 주요 키워드 TOP 20:")
    for i, (keyword, freq) in enumerate(sorted_keywords[:20], 1):
        print(f"   {i:2d}. {keyword}: {freq}회")
    
    # 긍정/부정 키워드 비율
    positive_total = sum(freq for keyword, freq in keyword_freq.items() if keyword in positive_keywords)
    negative_total = sum(freq for keyword, freq in keyword_freq.items() if keyword in negative_keywords)
    total_sentiment = positive_total + negative_total
    
    if total_sentiment > 0:
        positive_ratio = positive_total / total_sentiment * 100
        negative_ratio = negative_total / total_sentiment * 100
        
        print(f"\\n😊 감정 분석 결과:")
        print(f"   긍정 키워드: {positive_ratio:.1f}% ({positive_total}회)")
        print(f"   부정 키워드: {negative_ratio:.1f}% ({negative_total}회)")
    
    # 워드클라우드 생성
    create_wordcloud_visualization(keyword_freq)
    
    return keyword_freq

def create_wordcloud_visualization(keyword_freq):
    """워드클라우드 시각화 함수"""
    
    # WordCloud 라이브러리가 있으면 사용
    if 'WordCloud' in globals() and WordCloud is not None:
        try:
            # 한글 폰트 경로 설정
            font_paths = [
                'C:/Windows/Fonts/malgun.ttf',  # Windows
                '/System/Library/Fonts/AppleGothic.ttf',  # macOS
                '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'  # Linux
            ]
            
            font_path = None
            for path in font_paths:
                try:
                    with open(path, 'rb'):
                        font_path = path
                        break
                except:
                    continue
            
            # 워드클라우드 생성
            wordcloud = WordCloud(
                width=800, height=400,
                background_color='white',
                font_path=font_path,
                max_words=100,
                colormap='viridis'
            ).generate_from_frequencies(keyword_freq)
            
            plt.figure(figsize=(12, 6))
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.axis('off')
            plt.title('전통시장 리뷰 키워드 워드클라우드', fontsize=16, fontweight='bold')
            plt.tight_layout()
            plt.show()
            
            print("☁️ 워드클라우드 생성 완료!")
            
        except Exception as e:
            print(f"⚠️ 워드클라우드 생성 실패: {e}")
            create_alternative_keyword_visualization(keyword_freq)
    else:
        print("⚠️ WordCloud 라이브러리가 없습니다. 대체 시각화를 사용합니다.")
        create_alternative_keyword_visualization(keyword_freq)

def create_alternative_keyword_visualization(keyword_freq):
    """워드클라우드 대체 시각화"""
    
    # 상위 15개 키워드로 막대 그래프 생성
    sorted_keywords = sorted(keyword_freq.items(), key=lambda x: x[1], reverse=True)[:15]
    keywords, frequencies = zip(*sorted_keywords)
    
    plt.figure(figsize=(12, 8))
    bars = plt.barh(range(len(keywords)), frequencies, color='skyblue')
    plt.yticks(range(len(keywords)), keywords)
    plt.xlabel('빈도')
    plt.title('전통시장 리뷰 주요 키워드 (TOP 15)', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    
    # 막대에 값 표시
    for i, (bar, freq) in enumerate(zip(bars, frequencies)):
        plt.text(freq + max(frequencies) * 0.01, i, str(freq), 
                va='center', ha='left')
    
    plt.tight_layout()
    plt.show()
    
    print("📊 키워드 막대 그래프 생성 완료!")

# 키워드 분석 실행
keyword_results = keyword_analysis_simulation(market_df_final)


In [ ]:
# 5.2 정책 제안 생성 및 우선순위 분석

def generate_policy_recommendations(df, high_perf, low_perf, keyword_freq):
    """분석 결과를 바탕으로 정책 제안 생성"""
    
    print("🏛️ 전통시장 활성화 정책 제안")
    print("=" * 60)
    
    # 1. 단기 정책 (1년 이내)
    print("\\n📅 단기 정책 제안 (1년 이내)")
    print("-" * 40)
    
    short_term_policies = [
        {
            "제목": "시설 개선 우선순위 지원",
            "내용": "주차장, 화장실, 아케이드 등 기본 시설 개선 지원",
            "근거": "고성과 시장의 시설 보유율이 저성과 시장보다 높음",
            "예산": "시장당 평균 5,000만원",
            "대상": "시설 보유율이 낮은 하위 30% 시장"
        },
        {
            "제목": "디지털 마케팅 지원",
            "내용": "온라인 리뷰 관리 및 SNS 마케팅 교육 프로그램",
            "근거": "리뷰 수와 평점이 시장 성과와 상관관계 있음",
            "예산": "시장당 평균 500만원",
            "대상": "리뷰 수가 적은 시장 우선"
        },
        {
            "제목": "청결도 개선 캠페인",
            "내용": "정기적인 청소 및 위생 관리 시스템 구축",
            "근거": "키워드 분석에서 '깨끗하다'가 긍정 요인으로 나타남",
            "예산": "시장당 월 200만원",
            "대상": "전체 시장 대상"
        }
    ]
    
    for i, policy in enumerate(short_term_policies, 1):
        print(f"\\n{i}. {policy['제목']}")
        print(f"   📋 내용: {policy['내용']}")
        print(f"   📊 근거: {policy['근거']}")
        print(f"   💰 예산: {policy['예산']}")
        print(f"   🎯 대상: {policy['대상']}")
    
    # 2. 중기 정책 (2-3년)
    print("\\n\\n📅 중기 정책 제안 (2-3년)")
    print("-" * 40)
    
    medium_term_policies = [
        {
            "제목": "스마트 전통시장 구축",
            "내용": "IoT, 키오스크, 모바일 결제 시스템 도입",
            "근거": "접근성과 편리성이 고객 만족도에 영향",
            "예산": "시장당 평균 2억원",
            "대상": "중대형 시장 우선 (점포수 100개 이상)"
        },
        {
            "제목": "지역 특화 상품 개발",
            "내용": "지역별 특산품을 활용한 브랜드 상품 개발",
            "근거": "지역별 시장 성과 차이 존재",
            "예산": "지역당 평균 1억원",
            "대상": "지역 대표 시장 선정"
        },
        {
            "제목": "교통 접근성 개선",
            "내용": "대중교통 연계 및 주차 시설 확충",
            "근거": "접근성이 시장 이용률에 직접적 영향",
            "예산": "시장당 평균 3억원",
            "대상": "교통 불편 지역 시장 우선"
        }
    ]
    
    for i, policy in enumerate(medium_term_policies, 1):
        print(f"\\n{i}. {policy['제목']}")
        print(f"   📋 내용: {policy['내용']}")
        print(f"   📊 근거: {policy['근거']}")
        print(f"   💰 예산: {policy['예산']}")
        print(f"   🎯 대상: {policy['대상']}")
    
    # 3. 장기 정책 (3-5년)
    print("\\n\\n📅 장기 정책 제안 (3-5년)")
    print("-" * 40)
    
    long_term_policies = [
        {
            "제목": "전통시장 복합문화공간 조성",
            "내용": "쇼핑, 문화, 체험이 결합된 복합공간으로 재구성",
            "근거": "전통적 가치와 현대적 편의성의 조화 필요",
            "예산": "시장당 평균 10억원",
            "대상": "역사성과 잠재력이 높은 대표 시장"
        },
        {
            "제목": "세대 간 상인 육성 프로그램",
            "내용": "젊은 상인 유입 및 기존 상인과의 멘토링 시스템",
            "근거": "운영년수와 혁신성의 균형 필요",
            "예산": "연간 50억원 (전국 단위)",
            "대상": "전체 시장 대상"
        },
        {
            "제목": "글로벌 관광 명소화",
            "내용": "외국인 관광객 대상 K-마켓 브랜드 구축",
            "근거": "문화 관광 자원으로서의 가치 활용",
            "예산": "시장당 평균 5억원",
            "대상": "관광지 인근 대형 시장"
        }
    ]
    
    for i, policy in enumerate(long_term_policies, 1):
        print(f"\\n{i}. {policy['제목']}")
        print(f"   📋 내용: {policy['내용']}")
        print(f"   📊 근거: {policy['근거']}")
        print(f"   💰 예산: {policy['예산']}")
        print(f"   🎯 대상: {policy['대상']}")
    
    # 4. 우선순위 시장 선정
    print("\\n\\n🎯 우선 지원 대상 시장 선정")
    print("-" * 40)
    
    if not df.empty:
        # 개선 잠재력이 높은 시장 선정 (현재 성과는 낮지만 기본 조건이 좋은 시장)
        priority_markets = identify_priority_markets(df)
        
        print("\\n📊 우선 지원 대상 시장 (상위 10개):")
        for i, (market_name, score, reason) in enumerate(priority_markets[:10], 1):
            print(f"   {i:2d}. {market_name} (점수: {score:.2f})")
            print(f"       사유: {reason}")
    
    # 5. 예상 효과 및 성과 지표
    print("\\n\\n📈 예상 효과 및 성과 지표")
    print("-" * 40)
    
    expected_effects = [
        "평균 평점 0.5점 향상 (현재 대비 15% 증가)",
        "리뷰 수 30% 증가 (온라인 인지도 향상)",
        "시설 보유율 20%p 향상",
        "방문객 수 25% 증가",
        "매출액 20% 증가",
        "상인 만족도 30% 향상"
    ]
    
    for i, effect in enumerate(expected_effects, 1):
        print(f"   {i}. {effect}")
    
    print("\\n✅ 정책 제안 완료!")

def identify_priority_markets(df):
    """우선 지원 대상 시장 식별"""
    
    priority_list = []
    
    for idx, row in df.iterrows():
        score = 0
        reasons = []
        
        market_name = row.get('시장명', f'시장_{idx}')
        
        # 점포수가 많으면 가점 (규모의 경제)
        if '점포수' in row and pd.notna(row['점포수']):
            if row['점포수'] > 100:
                score += 2
                reasons.append("대규모 시장")
            elif row['점포수'] > 50:
                score += 1
                reasons.append("중규모 시장")
        
        # 평점이 중간 정도면 가점 (개선 여지)
        if '평점' in row and pd.notna(row['평점']):
            if 3.0 <= row['평점'] <= 4.0:
                score += 2
                reasons.append("개선 잠재력")
            elif row['평점'] < 3.0:
                score += 1
                reasons.append("개선 필요")
        
        # 시설이 부족하면 가점 (개선 효과 클 것으로 예상)
        facility_score = 0
        for facility in ['아케이드', '주차장', '화장실']:
            if facility in row and row[facility] == '없음':
                facility_score += 1
        
        if facility_score >= 2:
            score += 2
            reasons.append("시설 개선 필요")
        elif facility_score == 1:
            score += 1
            reasons.append("일부 시설 개선 필요")
        
        # 운영년수가 적당하면 가점 (너무 오래되지도 새롭지도 않은)
        if '운영년수' in row and pd.notna(row['운영년수']):
            if 10 <= row['운영년수'] <= 30:
                score += 1
                reasons.append("적정 운영 경험")
        
        if score > 0:
            reason_text = ", ".join(reasons)
            priority_list.append((market_name, score, reason_text))
    
    # 점수 순으로 정렬
    priority_list.sort(key=lambda x: x[1], reverse=True)
    
    return priority_list

# 정책 제안 생성
generate_policy_recommendations(market_df_final, high_perf_markets, low_perf_markets, keyword_results)


In [ ]:
# 5.3 데이터 저장 및 결과 파일 생성
# 분석 결과를 CSV 파일로 저장

def save_analysis_results(df, high_perf, low_perf, keyword_freq):
    """분석 결과를 CSV 파일로 저장"""
    
    print("💾 분석 결과 저장 중...")
    
    try:
        # 1. 전체 분석 데이터 저장
        if not df.empty:
            df.to_csv('전통시장_분석결과_전체.csv', index=False, encoding='utf-8-sig')
            print("✅ 전체 분석 데이터 저장 완료: 전통시장_분석결과_전체.csv")
        
        # 2. 고성과 시장 데이터 저장
        if not high_perf.empty:
            high_perf.to_csv('전통시장_고성과시장.csv', index=False, encoding='utf-8-sig')
            print("✅ 고성과 시장 데이터 저장 완료: 전통시장_고성과시장.csv")
        
        # 3. 저성과 시장 데이터 저장
        if not low_perf.empty:
            low_perf.to_csv('전통시장_저성과시장.csv', index=False, encoding='utf-8-sig')
            print("✅ 저성과 시장 데이터 저장 완료: 전통시장_저성과시장.csv")
        
        # 4. 키워드 분석 결과 저장
        if keyword_freq:
            keyword_df = pd.DataFrame(list(keyword_freq.items()), columns=['키워드', '빈도'])
            keyword_df = keyword_df.sort_values('빈도', ascending=False)
            keyword_df.to_csv('전통시장_키워드분석.csv', index=False, encoding='utf-8-sig')
            print("✅ 키워드 분석 결과 저장 완료: 전통시장_키워드분석.csv")
        
        # 5. 분석 요약 보고서 생성
        create_summary_report(df, high_perf, low_perf, keyword_freq)
        
        print("\\n📊 저장된 파일 목록:")
        print("   1. 전통시장_분석결과_전체.csv - 전체 분석 데이터")
        print("   2. 전통시장_고성과시장.csv - 상위 20% 시장 데이터")
        print("   3. 전통시장_저성과시장.csv - 하위 20% 시장 데이터")
        print("   4. 전통시장_키워드분석.csv - 리뷰 키워드 분석 결과")
        print("   5. 전통시장_분석요약보고서.csv - 주요 통계 요약")
        
    except Exception as e:
        print(f"❌ 파일 저장 중 오류 발생: {e}")

def create_summary_report(df, high_perf, low_perf, keyword_freq):
    """분석 요약 보고서 생성"""
    
    summary_data = []
    
    # 기본 통계
    if not df.empty:
        summary_data.append(['전체 시장 수', len(df)])
        
        if '평점' in df.columns:
            summary_data.append(['평균 평점', f"{df['평점'].mean():.2f}"])
            summary_data.append(['최고 평점', f"{df['평점'].max():.1f}"])
            summary_data.append(['최저 평점', f"{df['평점'].min():.1f}"])
        
        if '리뷰수' in df.columns:
            summary_data.append(['평균 리뷰수', f"{df['리뷰수'].mean():.1f}"])
            summary_data.append(['최대 리뷰수', f"{df['리뷰수'].max():.0f}"])
        
        if '점포수' in df.columns:
            summary_data.append(['평균 점포수', f"{df['점포수'].mean():.1f}"])
            summary_data.append(['최대 점포수', f"{df['점포수'].max():.0f}"])
        
        # 시설 보유율
        for facility in ['아케이드', '주차장', '화장실']:
            if facility in df.columns:
                rate = (df[facility] == '있음').mean() * 100
                summary_data.append([f'{facility} 보유율', f"{rate:.1f}%"])
        
        # 지역별 분포
        if '시도' in df.columns:
            top_region = df['시도'].value_counts().index[0]
            top_count = df['시도'].value_counts().iloc[0]
            summary_data.append(['최다 시장 지역', f"{top_region} ({top_count}개)"])
    
    # 고성과 vs 저성과 비교
    if not high_perf.empty and not low_perf.empty:
        summary_data.append(['고성과 시장 수', len(high_perf)])
        summary_data.append(['저성과 시장 수', len(low_perf)])
        
        if '평점' in high_perf.columns and '평점' in low_perf.columns:
            high_avg = high_perf['평점'].mean()
            low_avg = low_perf['평점'].mean()
            summary_data.append(['고성과 평균 평점', f"{high_avg:.2f}"])
            summary_data.append(['저성과 평균 평점', f"{low_avg:.2f}"])
            summary_data.append(['평점 차이', f"{high_avg - low_avg:.2f}"])
    
    # 키워드 분석 결과
    if keyword_freq:
        top_keyword = max(keyword_freq.items(), key=lambda x: x[1])
        summary_data.append(['최다 키워드', f"{top_keyword[0]} ({top_keyword[1]}회)"])
        summary_data.append(['총 키워드 수', len(keyword_freq)])
    
    # DataFrame으로 변환 후 저장
    summary_df = pd.DataFrame(summary_data, columns=['항목', '값'])
    summary_df.to_csv('전통시장_분석요약보고서.csv', index=False, encoding='utf-8-sig')
    print("✅ 분석 요약 보고서 저장 완료: 전통시장_분석요약보고서.csv")

# 분석 결과 저장 실행
save_analysis_results(market_df_final, high_perf_markets, low_perf_markets, keyword_results)


In [ ]:
# 2.5 샘플 리뷰 데이터 생성 (크롤링 대체)
# 실제 크롤링이 어려운 환경을 고려하여 현실적인 샘플 데이터를 생성

def create_sample_review_data(market_df):
    """
    시장별 리뷰 데이터 샘플 생성 함수
    시장 규모와 지역을 고려한 현실적인 데이터 생성
    
    Args:
        market_df (DataFrame): 시장 기본 정보 데이터프레임
    
    Returns:
        DataFrame: 리뷰 데이터가 추가된 데이터프레임
    """
    np.random.seed(42)  # 재현 가능한 결과
    
    # 시장 데이터 복사
    df = market_df.copy()
    
    # 시장 규모에 따른 리뷰 수 생성 (점포수와 상관관계 있음)
    base_reviews = df['점포수'] * np.random.uniform(0.5, 2.0, len(df))
    df['리뷰수'] = np.round(base_reviews).astype(int)
    
    # 시설 조건에 따른 평점 생성
    base_rating = 3.5  # 기본 평점
    
    # 주차장이 있으면 평점 상승
    parking_bonus = np.where(df['주차장유무'] == '있음', 0.3, -0.2)
    
    # 아케이드가 있으면 평점 상승 (날씨 영향 적음)
    arcade_bonus = np.where(df['아케이드유무'] == '있음', 0.2, -0.1)
    
    # 화장실 개수에 따른 평점 (많을수록 좋음)
    toilet_bonus = (df['화장실개수'] - df['화장실개수'].mean()) * 0.05
    
    # 개설년도에 따른 평점 (너무 오래되면 시설이 낡음)
    current_year = 2025
    age_penalty = np.where(current_year - df['개설년도'] > 50, -0.3, 0.1)
    
    # 최종 평점 계산 (1.0 ~ 5.0 범위)
    calculated_rating = base_rating + parking_bonus + arcade_bonus + toilet_bonus + age_penalty
    calculated_rating += np.random.normal(0, 0.3, len(df))  # 랜덤 노이즈 추가
    
    # 평점 범위 제한
    df['평점'] = np.clip(calculated_rating, 1.0, 5.0)
    df['평점'] = np.round(df['평점'], 1)
    
    # 일부 시장은 리뷰가 없을 수 있음 (소규모 시장)
    no_review_mask = (df['점포수'] < 80) & (np.random.random(len(df)) < 0.2)
    df.loc[no_review_mask, ['평점', '리뷰수']] = [None, 0]
    
    return df

# 리뷰 데이터 생성
market_with_reviews = create_sample_review_data(market_df)

print("📝 리뷰 데이터 생성 완료!")
print(f"평점 데이터가 있는 시장: {market_with_reviews['평점'].notna().sum()}개")
print(f"평균 평점: {market_with_reviews['평점'].mean():.2f}")
print(f"평균 리뷰수: {market_with_reviews['리뷰수'].mean():.0f}개")

print("\n🔍 리뷰 데이터 미리보기:")
print(market_with_reviews[['시장명', '지역', '점포수', '평점', '리뷰수']].head(10))


In [ ]:
# 2.6 데이터 전처리 및 정제

def preprocess_market_data(df):
    """
    시장 데이터 전처리 함수
    
    Args:
        df (DataFrame): 원본 데이터프레임
    
    Returns:
        DataFrame: 전처리된 데이터프레임
    """
    # 데이터 복사
    processed_df = df.copy()
    
    # 1. 시장명 표준화 (공백 제거, 일관성 확보)
    processed_df['시장명'] = processed_df['시장명'].str.strip()
    processed_df['시장명'] = processed_df['시장명'].str.replace('시장', '').str.replace('市場', '') + '시장'
    
    # 2. 결측값 처리
    # 평점이 없는 경우 지역 평균으로 대체
    for region in processed_df['지역'].unique():
        region_mask = processed_df['지역'] == region
        region_mean_rating = processed_df.loc[region_mask, '평점'].mean()
        
        if not pd.isna(region_mean_rating):
            processed_df.loc[region_mask & processed_df['평점'].isna(), '평점'] = region_mean_rating
    
    # 3. 새로운 파생 변수 생성
    # 시장 규모 분류
    processed_df['규모분류'] = pd.cut(processed_df['점포수'], 
                                   bins=[0, 100, 200, 300, float('inf')], 
                                   labels=['소형', '중형', '대형', '초대형'])
    
    # 시장 연령 계산
    current_year = 2025
    processed_df['시장연령'] = current_year - processed_df['개설년도']
    
    # 연령대 분류
    processed_df['연령대분류'] = pd.cut(processed_df['시장연령'], 
                                    bins=[0, 30, 50, 70, float('inf')], 
                                    labels=['신규', '중간', '오래됨', '매우오래됨'])
    
    # 4. 시설 점수 계산 (종합 시설 수준)
    facility_score = 0
    facility_score += np.where(processed_df['주차장유무'] == '있음', 25, 0)
    facility_score += np.where(processed_df['아케이드유무'] == '있음', 20, 0)
    facility_score += (processed_df['화장실개수'] / processed_df['화장실개수'].max()) * 30
    facility_score += np.where(processed_df['시장연령'] < 30, 25, 
                              np.where(processed_df['시장연령'] < 50, 15, 5))
    
    processed_df['시설점수'] = facility_score
    
    # 5. 수도권/지방 분류
    metropolitan_areas = ['서울', '인천', '경기']
    processed_df['수도권여부'] = processed_df['지역'].apply(
        lambda x: '수도권' if x in metropolitan_areas else '지방'
    )
    
    return processed_df

# 데이터 전처리 실행
final_market_data = preprocess_market_data(market_with_reviews)

print("🔧 데이터 전처리 완료!")
print(f"전처리된 데이터 크기: {final_market_data.shape}")
print("\n📊 전처리 결과 요약:")
print(f"- 규모분류별 시장 수: \n{final_market_data['규모분류'].value_counts()}")
print(f"\n- 수도권/지방별 시장 수: \n{final_market_data['수도권여부'].value_counts()}")
print(f"\n- 평균 시설점수: {final_market_data['시설점수'].mean():.1f}점")

# 전처리된 데이터 미리보기
print("\n🔍 전처리된 데이터 미리보기:")
display_columns = ['시장명', '지역', '규모분류', '평점', '시설점수', '수도권여부']
print(final_market_data[display_columns].head())


In [ ]:
# 3.3 기술통계 분석

def analyze_basic_statistics(df):
    """기본 통계 분석 함수"""
    
    print("📊 전국 전통시장 기본 현황 분석")
    print("=" * 50)
    
    # 1. 전체 현황
    print(f"🏪 총 분석 대상 시장 수: {len(df)}개")
    print(f"📍 분석 지역 수: {df['지역'].nunique()}개 지역")
    print(f"🛍️ 총 점포 수: {df['점포수'].sum():,}개")
    print(f"📏 총 시장 면적: {df['면적(㎡)'].sum():,}㎡")
    
    # 2. 평점 현황
    print(f"\n⭐ 평균 평점: {df['평점'].mean():.2f}점")
    print(f"📝 평균 리뷰 수: {df['리뷰수'].mean():.0f}개")
    print(f"🏆 최고 평점: {df['평점'].max():.1f}점")
    print(f"📉 최저 평점: {df['평점'].min():.1f}점")
    
    # 3. 시설 현황
    parking_rate = (df['주차장유무'] == '있음').mean() * 100
    arcade_rate = (df['아케이드유무'] == '있음').mean() * 100
    
    print(f"\n🚗 주차장 보유율: {parking_rate:.1f}%")
    print(f"🏢 아케이드 보유율: {arcade_rate:.1f}%")
    print(f"🚻 평균 화장실 개수: {df['화장실개수'].mean():.1f}개")
    print(f"📅 평균 시장 연령: {df['시장연령'].mean():.1f}년")
    
    # 4. 지역별 현황
    print(f"\n🗺️ 지역별 시장 분포:")
    region_stats = df.groupby('지역').agg({
        '시장명': 'count',
        '평점': 'mean',
        '점포수': 'mean'
    }).round(2)
    region_stats.columns = ['시장수', '평균평점', '평균점포수']
    print(region_stats.sort_values('평균평점', ascending=False))
    
    # 5. 규모별 현황
    print(f"\n📏 규모별 시장 분포:")
    size_stats = df.groupby('규모분류').agg({
        '시장명': 'count',
        '평점': 'mean',
        '시설점수': 'mean'
    }).round(2)
    size_stats.columns = ['시장수', '평균평점', '평균시설점수']
    print(size_stats)

# 기술통계 분석 실행
analyze_basic_statistics(final_market_data)


In [ ]:
# 4.2 시각화 분석 1: 평점 분포 및 지역별 비교

def create_rating_analysis_plots(df):
    """평점 관련 시각화 생성 함수"""
    
    # 그래프 크기 설정
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('전통시장 평점 분석', fontsize=16, fontweight='bold')
    
    # 1. 평점 분포 히스토그램
    axes[0, 0].hist(df['평점'].dropna(), bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].set_title('평점 분포')
    axes[0, 0].set_xlabel('평점')
    axes[0, 0].set_ylabel('시장 수')
    axes[0, 0].axvline(df['평점'].mean(), color='red', linestyle='--', 
                       label=f'평균: {df["평점"].mean():.2f}')
    axes[0, 0].legend()
    
    # 2. 지역별 평점 박스플롯
    if sns:
        sns.boxplot(data=df, x='지역', y='평점', ax=axes[0, 1])
        axes[0, 1].tick_params(axis='x', rotation=45)
    else:
        # seaborn이 없는 경우 대체 시각화
        region_ratings = [df[df['지역'] == region]['평점'].dropna().values 
                         for region in df['지역'].unique()]
        axes[0, 1].boxplot(region_ratings, labels=df['지역'].unique())
        axes[0, 1].tick_params(axis='x', rotation=45)
    
    axes[0, 1].set_title('지역별 평점 분포')
    axes[0, 1].set_ylabel('평점')
    
    # 3. 규모별 평점 비교
    size_ratings = df.groupby('규모분류')['평점'].mean().sort_values(ascending=False)
    bars = axes[1, 0].bar(size_ratings.index, size_ratings.values, 
                         color=['gold', 'silver', 'orange', 'lightcoral'])
    axes[1, 0].set_title('규모별 평균 평점')
    axes[1, 0].set_ylabel('평균 평점')
    
    # 막대 위에 값 표시
    for bar, value in zip(bars, size_ratings.values):
        axes[1, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                        f'{value:.2f}', ha='center', va='bottom')
    
    # 4. 시설점수와 평점의 관계
    axes[1, 1].scatter(df['시설점수'], df['평점'], alpha=0.6, color='green')
    axes[1, 1].set_title('시설점수 vs 평점')
    axes[1, 1].set_xlabel('시설점수')
    axes[1, 1].set_ylabel('평점')
    
    # 추세선 추가
    z = np.polyfit(df['시설점수'], df['평점'], 1)
    p = np.poly1d(z)
    axes[1, 1].plot(df['시설점수'], p(df['시설점수']), "r--", alpha=0.8)
    
    plt.tight_layout()
    plt.show()
    
    # 상관계수 계산
    correlation = df['시설점수'].corr(df['평점'])
    print(f"📈 시설점수와 평점의 상관계수: {correlation:.3f}")

# 평점 분석 시각화 실행
create_rating_analysis_plots(final_market_data)


In [ ]:
# 4.3 시각화 분석 2: 시설 현황 및 영향 요인 분석

def create_facility_analysis_plots(df):
    """시설 현황 관련 시각화 생성 함수"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('전통시장 시설 현황 분석', fontsize=16, fontweight='bold')
    
    # 1. 주차장 유무에 따른 평점 비교
    parking_ratings = df.groupby('주차장유무')['평점'].mean()
    bars1 = axes[0, 0].bar(parking_ratings.index, parking_ratings.values, 
                          color=['lightcoral', 'lightgreen'])
    axes[0, 0].set_title('주차장 유무별 평균 평점')
    axes[0, 0].set_ylabel('평균 평점')
    
    for bar, value in zip(bars1, parking_ratings.values):
        axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                        f'{value:.2f}', ha='center', va='bottom')
    
    # 2. 아케이드 유무에 따른 평점 비교
    arcade_ratings = df.groupby('아케이드유무')['평점'].mean()
    bars2 = axes[0, 1].bar(arcade_ratings.index, arcade_ratings.values, 
                          color=['lightcoral', 'lightblue'])
    axes[0, 1].set_title('아케이드 유무별 평균 평점')
    axes[0, 1].set_ylabel('평균 평점')
    
    for bar, value in zip(bars2, arcade_ratings.values):
        axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                        f'{value:.2f}', ha='center', va='bottom')
    
    # 3. 시장 연령대별 평점 분포
    age_ratings = df.groupby('연령대분류')['평점'].mean()
    bars3 = axes[1, 0].bar(age_ratings.index, age_ratings.values, 
                          color=['gold', 'orange', 'red', 'darkred'])
    axes[1, 0].set_title('시장 연령대별 평균 평점')
    axes[1, 0].set_ylabel('평균 평점')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    for bar, value in zip(bars3, age_ratings.values):
        axes[1, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                        f'{value:.2f}', ha='center', va='bottom')
    
    # 4. 수도권/지방별 시설점수 비교
    metro_facility = df.groupby('수도권여부')['시설점수'].mean()
    bars4 = axes[1, 1].bar(metro_facility.index, metro_facility.values, 
                          color=['purple', 'teal'])
    axes[1, 1].set_title('수도권/지방별 평균 시설점수')
    axes[1, 1].set_ylabel('평균 시설점수')
    
    for bar, value in zip(bars4, metro_facility.values):
        axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                        f'{value:.1f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

# 시설 분석 시각화 실행
create_facility_analysis_plots(final_market_data)


In [ ]:
# 4.4 상관관계 분석 및 히트맵

def analyze_correlations(df):
    """상관관계 분석 및 시각화 함수"""
    
    # 수치형 변수들 선택
    numeric_columns = ['점포수', '면적(㎡)', '화장실개수', '시장연령', '평점', '리뷰수', '시설점수']
    correlation_data = df[numeric_columns].corr()
    
    # 상관관계 히트맵 생성
    plt.figure(figsize=(10, 8))
    
    if sns:
        # seaborn이 있는 경우
        sns.heatmap(correlation_data, annot=True, cmap='coolwarm', center=0,
                   square=True, fmt='.3f', cbar_kws={'shrink': 0.8})
    else:
        # seaborn이 없는 경우 matplotlib로 대체
        im = plt.imshow(correlation_data, cmap='coolwarm', aspect='auto')
        plt.colorbar(im, shrink=0.8)
        
        # 상관계수 값 표시
        for i in range(len(correlation_data.columns)):
            for j in range(len(correlation_data.columns)):
                plt.text(j, i, f'{correlation_data.iloc[i, j]:.3f}',
                        ha='center', va='center', color='black')
        
        plt.xticks(range(len(correlation_data.columns)), correlation_data.columns, rotation=45)
        plt.yticks(range(len(correlation_data.columns)), correlation_data.columns)
    
    plt.title('전통시장 변수 간 상관관계 분석', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # 주요 상관관계 분석 결과 출력
    print("🔍 주요 상관관계 분석 결과:")
    print("=" * 40)
    
    # 평점과 다른 변수들의 상관관계
    rating_corr = correlation_data['평점'].sort_values(ascending=False)
    print("📊 평점과의 상관관계:")
    for var, corr in rating_corr.items():
        if var != '평점':
            print(f"  - {var}: {corr:.3f}")
    
    # 강한 상관관계 (절댓값 0.5 이상) 찾기
    print(f"\n💪 강한 상관관계 (|r| ≥ 0.5):")
    strong_corr = []
    for i in range(len(correlation_data.columns)):
        for j in range(i+1, len(correlation_data.columns)):
            corr_value = correlation_data.iloc[i, j]
            if abs(corr_value) >= 0.5:
                var1 = correlation_data.columns[i]
                var2 = correlation_data.columns[j]
                strong_corr.append((var1, var2, corr_value))
    
    for var1, var2, corr in strong_corr:
        print(f"  - {var1} ↔ {var2}: {corr:.3f}")
    
    if not strong_corr:
        print("  - 강한 상관관계를 보이는 변수 쌍이 없습니다.")

# 상관관계 분석 실행
analyze_correlations(final_market_data)


In [ ]:
# 4.5 키워드 분석 및 워드클라우드

def create_keyword_analysis(df):
    """키워드 분석 및 워드클라우드 생성 함수"""
    
    # 전통시장 관련 키워드 생성 (실제로는 리뷰 텍스트에서 추출)
    # 여기서는 시장 특성을 반영한 샘플 키워드를 생성
    
    # 시장별 특성에 따른 키워드 생성
    keywords_data = []
    
    for _, market in df.iterrows():
        # 기본 키워드
        base_keywords = ['전통시장', '쇼핑', '구매']
        
        # 평점에 따른 키워드
        if market['평점'] >= 4.0:
            positive_keywords = ['좋음', '만족', '추천', '친절', '신선', '저렴']
            base_keywords.extend(positive_keywords)
        elif market['평점'] >= 3.0:
            neutral_keywords = ['보통', '괜찮음', '적당']
            base_keywords.extend(neutral_keywords)
        else:
            negative_keywords = ['아쉬움', '개선필요', '불편']
            base_keywords.extend(negative_keywords)
        
        # 시설에 따른 키워드
        if market['주차장유무'] == '있음':
            base_keywords.extend(['주차편리', '접근성'])
        else:
            base_keywords.extend(['주차불편'])
            
        if market['아케이드유무'] == '있음':
            base_keywords.extend(['실내', '날씨무관'])
        else:
            base_keywords.extend(['야외', '날씨영향'])
        
        # 주요상품에 따른 키워드
        if market['주요상품'] == '식품':
            base_keywords.extend(['음식', '먹거리', '맛집'])
        elif market['주요상품'] == '의류':
            base_keywords.extend(['옷', '패션', '의류'])
        elif market['주요상품'] == '수산물':
            base_keywords.extend(['생선', '해산물', '신선'])
        
        keywords_data.extend(base_keywords)
    
    # 키워드 빈도 계산
    from collections import Counter
    keyword_freq = Counter(keywords_data)
    
    # 상위 키워드 출력
    print("🔤 전통시장 관련 주요 키워드 분석")
    print("=" * 40)
    print("📊 상위 10개 키워드:")
    
    top_keywords = keyword_freq.most_common(10)
    for i, (keyword, freq) in enumerate(top_keywords, 1):
        print(f"{i:2d}. {keyword}: {freq}회")
    
    # 워드클라우드 생성
    if WordCloud:
        # 한글 폰트 경로 설정 (Windows 기준)
        font_path = 'C:/Windows/Fonts/malgun.ttf'  # 맑은 고딕
        
        try:
            wordcloud = WordCloud(
                font_path=font_path,
                width=800, 
                height=400,
                background_color='white',
                max_words=50,
                colormap='viridis'
            ).generate_from_frequencies(keyword_freq)
            
            plt.figure(figsize=(12, 6))
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.axis('off')
            plt.title('전통시장 키워드 워드클라우드', fontsize=16, fontweight='bold')
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            print(f"⚠️ 워드클라우드 생성 실패: {e}")
            print("📊 대신 키워드 빈도 막대그래프를 생성합니다.")
            create_keyword_bar_chart(top_keywords)
    else:
        print("📊 WordCloud 라이브러리가 없어 막대그래프로 대체합니다.")
        create_keyword_bar_chart(top_keywords)

def create_keyword_bar_chart(top_keywords):
    """키워드 빈도 막대그래프 생성 함수"""
    keywords, frequencies = zip(*top_keywords)
    
    plt.figure(figsize=(12, 6))
    bars = plt.bar(keywords, frequencies, color='skyblue', alpha=0.7)
    plt.title('전통시장 주요 키워드 빈도', fontsize=14, fontweight='bold')
    plt.xlabel('키워드')
    plt.ylabel('빈도')
    plt.xticks(rotation=45)
    
    # 막대 위에 빈도 표시
    for bar, freq in zip(bars, frequencies):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                str(freq), ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

# 키워드 분석 실행
create_keyword_analysis(final_market_data)


In [ ]:
# 4.6 종합 인사이트 도출

def generate_insights(df):
    """데이터 분석을 통한 종합 인사이트 도출 함수"""
    
    print("💡 전통시장 분석 주요 인사이트")
    print("=" * 50)
    
    # 1. 평점 관련 인사이트
    high_rating_markets = df[df['평점'] >= 4.0]
    low_rating_markets = df[df['평점'] < 3.0]
    
    print("🏆 1. 고평점 시장의 특징:")
    if len(high_rating_markets) > 0:
        parking_rate_high = (high_rating_markets['주차장유무'] == '있음').mean() * 100
        arcade_rate_high = (high_rating_markets['아케이드유무'] == '있음').mean() * 100
        avg_facility_high = high_rating_markets['시설점수'].mean()
        
        print(f"   - 고평점 시장 수: {len(high_rating_markets)}개")
        print(f"   - 주차장 보유율: {parking_rate_high:.1f}%")
        print(f"   - 아케이드 보유율: {arcade_rate_high:.1f}%")
        print(f"   - 평균 시설점수: {avg_facility_high:.1f}점")
    
    print(f"\n📉 2. 저평점 시장의 특징:")
    if len(low_rating_markets) > 0:
        parking_rate_low = (low_rating_markets['주차장유무'] == '있음').mean() * 100
        arcade_rate_low = (low_rating_markets['아케이드유무'] == '있음').mean() * 100
        avg_facility_low = low_rating_markets['시설점수'].mean()
        avg_age_low = low_rating_markets['시장연령'].mean()
        
        print(f"   - 저평점 시장 수: {len(low_rating_markets)}개")
        print(f"   - 주차장 보유율: {parking_rate_low:.1f}%")
        print(f"   - 아케이드 보유율: {arcade_rate_low:.1f}%")
        print(f"   - 평균 시설점수: {avg_facility_low:.1f}점")
        print(f"   - 평균 시장연령: {avg_age_low:.1f}년")
    
    # 2. 지역별 인사이트
    print(f"\n🗺️ 3. 지역별 특성:")
    region_analysis = df.groupby('지역').agg({
        '평점': 'mean',
        '시설점수': 'mean',
        '시장연령': 'mean'
    }).round(2)
    
    best_region = region_analysis['평점'].idxmax()
    worst_region = region_analysis['평점'].idxmin()
    
    print(f"   - 최고 평점 지역: {best_region} ({region_analysis.loc[best_region, '평점']:.2f}점)")
    print(f"   - 최저 평점 지역: {worst_region} ({region_analysis.loc[worst_region, '평점']:.2f}점)")
    
    # 3. 시설 개선 우선순위
    print(f"\n🔧 4. 시설 개선 우선순위:")
    
    # 주차장이 없는 시장 중 평점이 낮은 곳
    no_parking_low_rating = df[(df['주차장유무'] == '없음') & (df['평점'] < 3.5)]
    print(f"   - 주차장 설치 필요 시장: {len(no_parking_low_rating)}개")
    
    # 아케이드가 없는 시장 중 평점이 낮은 곳
    no_arcade_low_rating = df[(df['아케이드유무'] == '없음') & (df['평점'] < 3.5)]
    print(f"   - 아케이드 설치 필요 시장: {len(no_arcade_low_rating)}개")
    
    # 오래된 시장 중 평점이 낮은 곳
    old_low_rating = df[(df['시장연령'] > 50) & (df['평점'] < 3.5)]
    print(f"   - 시설 현대화 필요 시장: {len(old_low_rating)}개")
    
    # 4. 성공 요인 분석
    print(f"\n✨ 5. 성공 요인 분석:")
    
    # 시설점수와 평점의 상관관계
    facility_rating_corr = df['시설점수'].corr(df['평점'])
    print(f"   - 시설점수와 평점 상관계수: {facility_rating_corr:.3f}")
    
    if facility_rating_corr > 0.3:
        print("   → 시설 개선이 고객 만족도 향상에 중요한 역할을 함")
    
    # 규모와 평점의 관계
    size_rating = df.groupby('규모분류')['평점'].mean()
    best_size = size_rating.idxmax()
    print(f"   - 가장 높은 평점을 받는 시장 규모: {best_size}")
    
    # 5. 개선 권장사항
    print(f"\n📋 6. 개선 권장사항:")
    
    improvement_candidates = df[
        (df['평점'] < 3.5) & 
        ((df['주차장유무'] == '없음') | (df['아케이드유무'] == '없음') | (df['시장연령'] > 40))
    ].sort_values('평점')
    
    if len(improvement_candidates) > 0:
        print(f"   - 우선 개선 대상 시장: {len(improvement_candidates)}개")
        print("   - 상위 5개 개선 대상:")
        for i, (idx, market) in enumerate(improvement_candidates.head().iterrows(), 1):
            issues = []
            if market['주차장유무'] == '없음':
                issues.append('주차장')
            if market['아케이드유무'] == '없음':
                issues.append('아케이드')
            if market['시장연령'] > 40:
                issues.append('노후화')
            
            print(f"     {i}. {market['시장명']} (평점: {market['평점']:.1f}, 개선필요: {', '.join(issues)})")

# 종합 인사이트 생성
generate_insights(final_market_data)


In [ ]:
# 5.2 구체적 정책 제안 및 우선순위

def generate_policy_recommendations(df):
    """데이터 분석 결과를 바탕으로 한 정책 제안 생성 함수"""
    
    print("🏛️ 전통시장 활성화를 위한 정책 제안")
    print("=" * 60)
    
    # 분석 결과 요약
    total_markets = len(df)
    low_rating_markets = len(df[df['평점'] < 3.5])
    no_parking_markets = len(df[df['주차장유무'] == '없음'])
    no_arcade_markets = len(df[df['아케이드유무'] == '없음'])
    old_markets = len(df[df['시장연령'] > 50])
    
    print("📊 현황 요약:")
    print(f"   - 전체 분석 대상 시장: {total_markets}개")
    print(f"   - 개선 필요 시장 (평점 3.5 미만): {low_rating_markets}개 ({low_rating_markets/total_markets*100:.1f}%)")
    print(f"   - 주차장 미보유 시장: {no_parking_markets}개 ({no_parking_markets/total_markets*100:.1f}%)")
    print(f"   - 아케이드 미보유 시장: {no_arcade_markets}개 ({no_arcade_markets/total_markets*100:.1f}%)")
    print(f"   - 노후 시장 (50년 이상): {old_markets}개 ({old_markets/total_markets*100:.1f}%)")
    
    print(f"\n🎯 1. 단기 정책 제안 (1-2년)")
    print("-" * 40)
    
    # 단기 정책 1: 주차장 확충
    urgent_parking = df[(df['주차장유무'] == '없음') & (df['평점'] < 3.5)].sort_values('평점')
    print(f"📍 1-1. 주차장 확충 사업")
    print(f"   - 대상: 주차장 미보유 + 저평점 시장 {len(urgent_parking)}개")
    print(f"   - 예상 효과: 접근성 개선으로 고객 만족도 0.3-0.5점 향상 예상")
    print(f"   - 우선 대상 시장:")
    for i, (_, market) in enumerate(urgent_parking.head(3).iterrows(), 1):
        print(f"     {i}. {market['시장명']} (현재 평점: {market['평점']:.1f})")
    
    # 단기 정책 2: 디지털 마케팅 지원
    print(f"\n📱 1-2. 디지털 마케팅 지원 사업")
    low_review_markets = df[df['리뷰수'] < df['리뷰수'].median()].sort_values('리뷰수')
    print(f"   - 대상: 온라인 노출도가 낮은 시장 {len(low_review_markets)}개")
    print(f"   - 내용: 네이버/카카오맵 등록, SNS 마케팅 교육, 온라인 홍보 지원")
    print(f"   - 예상 효과: 온라인 인지도 향상으로 방문객 20-30% 증가")
    
    print(f"\n🏗️ 2. 중기 정책 제안 (3-5년)")
    print("-" * 40)
    
    # 중기 정책 1: 아케이드 설치
    urgent_arcade = df[(df['아케이드유무'] == '없음') & (df['평점'] < 4.0)].sort_values('평점')
    print(f"🏢 2-1. 아케이드 및 실내화 사업")
    print(f"   - 대상: 아케이드 미보유 시장 {len(urgent_arcade)}개")
    print(f"   - 내용: 지붕 설치, 실내 환경 조성, 냉난방 시설 구축")
    print(f"   - 예상 효과: 날씨 영향 최소화로 연중 안정적 운영")
    
    # 중기 정책 2: 시설 현대화
    old_low_rating = df[(df['시장연령'] > 40) & (df['평점'] < 3.5)]
    print(f"\n🔧 2-2. 시설 현대화 사업")
    print(f"   - 대상: 노후 + 저평점 시장 {len(old_low_rating)}개")
    print(f"   - 내용: 화장실 개선, 바닥 정비, 조명 교체, 안전시설 보강")
    print(f"   - 예상 효과: 쾌적한 환경 조성으로 고객 만족도 향상")
    
    print(f"\n🌟 3. 장기 정책 제안 (5-10년)")
    print("-" * 40)
    
    # 장기 정책 1: 특화 시장 육성
    print(f"🎨 3-1. 특화 시장 육성 사업")
    specialty_candidates = df[df['평점'] >= 4.0].sort_values('평점', ascending=False)
    print(f"   - 대상: 고평점 시장을 중심으로 특화 브랜드 개발")
    print(f"   - 내용: 지역 특산품, 문화 콘텐츠와 연계한 테마 시장 조성")
    print(f"   - 우수 시장 활용:")
    for i, (_, market) in enumerate(specialty_candidates.head(3).iterrows(), 1):
        print(f"     {i}. {market['시장명']} (평점: {market['평점']:.1f}, 주요상품: {market['주요상품']})")
    
    # 장기 정책 2: 스마트 시장 구축
    print(f"\n💻 3-2. 스마트 시장 구축 사업")
    print(f"   - 내용: IoT 기반 시설 관리, 모바일 결제 시스템, 배송 서비스")
    print(f"   - 대상: 대형 시장부터 단계적 확산")
    large_markets = df[df['규모분류'] == '대형'].sort_values('평점', ascending=False)
    print(f"   - 1단계 대상 (대형 시장): {len(large_markets)}개")
    
    print(f"\n💰 4. 예산 및 우선순위")
    print("-" * 40)
    
    # 비용 효과 분석
    print(f"📈 4-1. 투자 우선순위 (비용 대비 효과)")
    print(f"   1순위: 주차장 확충 (저비용, 즉시 효과)")
    print(f"   2순위: 디지털 마케팅 지원 (저비용, 단기 효과)")
    print(f"   3순위: 시설 현대화 (중비용, 중기 효과)")
    print(f"   4순위: 아케이드 설치 (고비용, 장기 효과)")
    
    # 예상 예산
    print(f"\n💵 4-2. 예상 예산 (개략적 추정)")
    print(f"   - 주차장 확충: 시장당 2-5억원 × {len(urgent_parking)}개 = {len(urgent_parking)*3:.0f}억원")
    print(f"   - 디지털 마케팅: 시장당 500만원 × {total_markets}개 = {total_markets*0.05:.0f}억원")
    print(f"   - 시설 현대화: 시장당 5-10억원 × {len(old_low_rating)}개 = {len(old_low_rating)*7:.0f}억원")
    print(f"   - 아케이드 설치: 시장당 10-20억원 × {len(urgent_arcade)}개 = {len(urgent_arcade)*15:.0f}억원")
    
    print(f"\n📋 5. 성과 지표 및 모니터링")
    print("-" * 40)
    print(f"   - 고객 만족도: 평균 평점 {df['평점'].mean():.2f}점 → 4.0점 목표")
    print(f"   - 방문객 증가율: 리뷰 수 기준 연 20% 증가 목표")
    print(f"   - 시설 개선율: 주차장 보유율 70% → 90% 목표")
    print(f"   - 디지털화율: 온라인 리뷰 보유 시장 100% 달성")
    
    return {
        'urgent_parking': urgent_parking,
        'low_review_markets': low_review_markets,
        'old_low_rating': old_low_rating,
        'specialty_candidates': specialty_candidates
    }

# 정책 제안 생성
policy_data = generate_policy_recommendations(final_market_data)


In [ ]:
# 6. 데이터 저장 (CSV 파일 생성)

def save_analysis_results(df, policy_data):
    """분석 결과를 CSV 파일로 저장하는 함수"""
    
    print("💾 분석 결과 데이터 저장")
    print("=" * 30)
    
    try:
        # 1. 전체 분석 데이터 저장
        final_filename = '전통시장_분석결과_최종.csv'
        df.to_csv(final_filename, index=False, encoding='utf-8-sig')
        print(f"✅ 전체 분석 데이터 저장: {final_filename}")
        
        # 2. 정책 대상 시장 데이터 저장
        if len(policy_data['urgent_parking']) > 0:
            parking_filename = '주차장_확충_대상시장.csv'
            policy_data['urgent_parking'].to_csv(parking_filename, index=False, encoding='utf-8-sig')
            print(f"✅ 주차장 확충 대상 저장: {parking_filename}")
        
        if len(policy_data['old_low_rating']) > 0:
            renovation_filename = '시설_현대화_대상시장.csv'
            policy_data['old_low_rating'].to_csv(renovation_filename, index=False, encoding='utf-8-sig')
            print(f"✅ 시설 현대화 대상 저장: {renovation_filename}")
        
        # 3. 우수 시장 데이터 저장
        if len(policy_data['specialty_candidates']) > 0:
            specialty_filename = '특화시장_육성_후보.csv'
            policy_data['specialty_candidates'].to_csv(specialty_filename, index=False, encoding='utf-8-sig')
            print(f"✅ 특화시장 후보 저장: {specialty_filename}")
        
        # 4. 요약 통계 저장
        summary_stats = {
            '구분': ['전체 시장 수', '평균 평점', '평균 리뷰수', '주차장 보유율(%)', '아케이드 보유율(%)', 
                   '평균 시설점수', '개선 필요 시장 수', '고평점 시장 수(4.0 이상)'],
            '값': [
                len(df),
                round(df['평점'].mean(), 2),
                round(df['리뷰수'].mean(), 0),
                round((df['주차장유무'] == '있음').mean() * 100, 1),
                round((df['아케이드유무'] == '있음').mean() * 100, 1),
                round(df['시설점수'].mean(), 1),
                len(df[df['평점'] < 3.5]),
                len(df[df['평점'] >= 4.0])
            ]
        }
        
        summary_df = pd.DataFrame(summary_stats)
        summary_filename = '분석_요약통계.csv'
        summary_df.to_csv(summary_filename, index=False, encoding='utf-8-sig')
        print(f"✅ 요약 통계 저장: {summary_filename}")
        
        print(f"\n📁 저장된 파일 목록:")
        print(f"   1. {final_filename} - 전체 분석 결과")
        print(f"   2. {summary_filename} - 요약 통계")
        print(f"   3. {parking_filename} - 주차장 확충 대상")
        print(f"   4. {renovation_filename} - 시설 현대화 대상")
        print(f"   5. {specialty_filename} - 특화시장 후보")
        
        return True
        
    except Exception as e:
        print(f"❌ 파일 저장 중 오류 발생: {e}")
        return False

# 데이터 저장 실행
save_success = save_analysis_results(final_market_data, policy_data)
